In [1]:
from data_loaders import Hip_Dataset,Hip_Dataset_Specific_Path, Hip_Dataset_Selected,Hip_Dataset_One_Complete
from PIL import Image
import train as Train
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from datetime import datetime
import pandas as pd
from Models import *
import matplotlib.pyplot as plt
from importlib import reload
import data_loaders
from sklearn.metrics import confusion_matrix
import numpy as np

In [2]:
Fracture_record=pd.read_csv("Fracture_record.csv")
image_size=(640,640)
Fracture_record.head(5)

,Unnamed: 0,Filename,Grid
0,0,Combined/10001502925_517424641.png,3
1,1,Combined/10001502925_517424641.png,3
2,2,Combined/10001083178_493490223.png,0
3,3,Combined/10001502925_517424641.png,3
4,4,Combined/10001541755_518066834.png,7


In [3]:
files =set(Fracture_record["Filename"])
print(files)

{'Batch_07/10001623568_523559394.png', 'Batch_06/10001574857_520308172.png', 'Batch_07/10001632103_524219578.png', 'datasets/10000914018_485782390.png', 'datasets/10000925240_486086363.png', 'Combined/10001226739_504239598.png', 'Batch_06/10001577733_520412474.png', 'datasets/10000921001_485910527.png', 'Combined/10001328116_511308725.png', 'Batch_07/10001608815_522630051.png', 'Batch_06/10001557255_519150117.png', 'Combined/10001502925_517424641.png', 'Batch_06/10001592839_521469374.png', 'Batch_07/10001610466_522753508.png', 'Batch_06/10001566440_519715224.png', 'Combined/10001083178_493490223.png', 'Batch_07/10001628722_524019683.png', 'Combined/10001541755_518066834.png', 'Combined/10000984194_487856846.png', 'Batch_07/10001601387_521972842.png', 'Combined/10001001739_488767941.png', 'Combined/10001272349_508125020.png', 'Combined/10001304082_509706236.png', 'Batch_06/10001563545_519545527.png'}


In [4]:
ann_path1 = 'Batch02/via_project_28Mar2021_16h4m_csv.csv'
ann_path="datasets/via_project_19Mar2021_23h45m_csv.csv"
ann_path2="Combined/list_final.csv"
ann_path3="Batch_06/batch_06.csv"
ann_path4="Batch_07/batch_07.csv"
ann_path5="Batch_08/batch_08.csv"

In [9]:
index=9
ld_test= Hip_Dataset(ann_path,"datasets",image_size,index)
ld_test2=Hip_Dataset(ann_path1,"Batch02",image_size,index)

ld_total= Hip_Dataset(ann_path2,"Combined",image_size,index)
ld_06= Hip_Dataset(ann_path3,"Batch_06",image_size,index)
ld_07= Hip_Dataset(ann_path4,"Batch_07",image_size,index)
ld_08= Hip_Dataset(ann_path5,"Batch_08",image_size,index)

In [10]:
total_loader=ld_test+ld_total+ld_06+ld_07+ld_08

In [11]:
len(total_loader)

11421

In [12]:
test_loader=DataLoader(ld_test, batch_size =4)
net= Custom_Net(240)

In [19]:
for img, label in test_loader:
    print(label.data)
    break

tensor([0])


In [17]:
y_true=[]
y_pred=[]
for img, label in test_loader:
    log_prob= net(img)[0]
    pred= torch.argmax(log_prob).numpy()
    y_t =label[0].numpy()
    y_true.append(y_t)
    y_pred.append(pred)
    
print(confusion_matrix(y_pred,y_true))

[[33 14]
 [29 17]]


In [7]:
Fracture_record=pd.read_csv("Fracture_record.csv")
index_count =Fracture_record["Grid"].value_counts()

In [ ]:
total_loaders=[ld_test,ld_total,ld_06,ld_07,ld_08]

In [13]:
def get_fracture_df(ld_combined):
    columns =["Filename", "Grid"]
    Fracture_record= pd.DataFrame(columns=columns)
    for ld in ld_combined:
        for k, (img, label) in enumerate(ld):
            if label==1:
                grid=k%9
                file_name=ld.get_file_name(k)
                Fracture_record=Fracture_record.append({"Filename":file_name,"Grid":grid},ignore_index = True)

    return Fracture_record
record =get_fracture_df([ld_test,ld_test2])

In [14]:
filenames=record["Filename"]


In [16]:
files_set =set(list(Fracture_record["Filename"])+(list(filenames)))
result = pd.concat([record, Fracture_record], axis=0)
len(result["Filename"])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


405

In [17]:
newdf=pd.DataFrame({"Filename" : list(files_set)})
newdf.to_csv("Fracture_image.csv")

In [5]:
newdf=pd.read_csv("Fracture_image.csv")

In [6]:

image_size=(640,640)

In [7]:
ld_test =Hip_Dataset_One_Complete(ann_path,"datasets",image_size)
ld_test2=Hip_Dataset_One_Complete(ann_path1,"Batch02",image_size)

ld_total= Hip_Dataset_One_Complete(ann_path2,"Combined",image_size)
ld_06= Hip_Dataset_One_Complete(ann_path3,"Batch_06",image_size)
ld_07= Hip_Dataset_One_Complete(ann_path4,"Batch_07",image_size)
ld_08= Hip_Dataset_One_Complete(ann_path5,"Batch_08",image_size)
total_loader=ld_test2+ld_total+ld_06+ld_07+ld_08
print(len(total_loader))

1322


In [8]:
augumentation_factor= len(total_loader)//107
ld_test3=Hip_Dataset_Selected("Fracture_image.csv",(640,640), augmentation=augumentation_factor)

In [9]:
total_train_loader=total_loader+ld_test3

In [10]:
for img, label in total_train_loader:
    print(img.shape)
    break

torch.Size([3, 640, 640])


In [13]:
epochs= 10
grid_indexes= [3,4,5,7]
weights=np.zeros(2)
weights=torch.FloatTensor(weights)
weights[0]=1
weights[1]=1
ann_path1 = 'Batch02/via_project_28Mar2021_16h4m_csv.csv'
ann_path="datasets/via_project_19Mar2021_23h45m_csv.csv"
ann_path2="Combined/list_final.csv"
ann_path3="Batch_06/batch_06.csv"
ann_path4="Batch_07/batch_07.csv"
ann_path5="Batch_08/batch_08.csv"


In [ ]:
train_loader=DataLoader(total_train_loader, batch_size = 10, shuffle = True)

model,val_accuracy,val_loss_list= Train.training_setup("googlenet",train_loader,ld_test2+ld_test, image_size[1],epochs,weights,index=0)